In [85]:
#libraries
import pandas as pd
import numpy as np
import sys
import os

from lifelines.utils import concordance_index
from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest, GradientBoostingSurvivalAnalysis
from sksurv.svm import FastKernelSurvivalSVM
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sklearn.impute import KNNImputer
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow import keras as ks
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize
from sklearn.linear_model import Lasso
from sksurv.util import Surv

In [38]:
def fillNA(df, k_neighbors=10, index=[]):
    if not list(index):
        index = df.index
    
    X = df.values
    knn = KNNImputer(n_neighbors=k_neighbors)
    X_imp = knn.fit_transform(X)
    
    df_imp = pd.DataFrame(X_imp, columns=df.columns)
    df_imp.set_index(index, inplace=True)
    
    return df_imp

def minmax_scale(train, test):
    xtrain_scaled = pd.DataFrame(MinMaxScaler().fit_transform(train), columns=train.columns)
    xtest_scaled = pd.DataFrame(MinMaxScaler().fit_transform(test), columns=test.columns)
    return xtrain_scaled, xtest_scaled  

def split_and_scale_data(features, labels, test_size=0.3):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=5, stratify=labels)
    X_train_scaled, X_test_scaled = minmax_scale(X_train, X_test)
    X_train_scaled.set_index(X_train.index, inplace=True)
    X_test_scaled.set_index(X_test.index, inplace=True)

    return X_train_scaled, X_test_scaled, y_train, y_test

def make_structured_array(event, time):
    return Surv.from_arrays(event, time)

class Autoencoder(ks.models.Model):
    def __init__(self, actual_dim, latent_dim, activation, loss, optimizer):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim

        self.encoder = ks.Sequential([
        ks.layers.Flatten(),
        ks.layers.Dense(latent_dim, activation=activation),
        ])

        self.decoder = ks.Sequential([
        ks.layers.Dense(actual_dim, activation=activation),
        ])

        self.compile(loss=loss, optimizer=optimizer, metrics=[ks.metrics.BinaryAccuracy(name='accuracy')])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

def create_AE(actual_dim=1, latent_dim=100, activation='relu', loss='MAE', optimizer='Adam'):
    return Autoencoder(actual_dim, latent_dim, activation, loss, optimizer)

def run_AE(X_train_scaled, X_test_scaled, param_grid=None):

    if param_grid == None:
        param_grid = {
            'actual_dim' : [len(X_train_scaled.columns)],
            'latent_dim' : [100, 200, 500],
            'activation' : ['relu', 'sigmoid', 'tanh'],
            'loss' : ['MAE', 'binary_crossentropy'],
            'optimizer' : ['SGD', 'Adam']
        }

    model = KerasClassifier(build_fn=create_AE, epochs=10, verbose=0)
    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=5,
        verbose=3
    )

    result = grid.fit(X_train_scaled, X_train_scaled, validation_data=(X_test_scaled, X_test_scaled))
    params = grid.best_params_
    autoencoder = create_AE(**params)

    try:
        encoder_layer = autoencoder.encoder
    except:
        exit

    AE_train = pd.DataFrame(encoder_layer.predict(X_train_scaled))
    AE_train.add_prefix('feature_')
    AE_test = pd.DataFrame(encoder_layer.predict(X_test_scaled))
    AE_test.add_prefix('feature_')

    return AE_train, AE_test

def run_LASSO(X_train_scaled, y_train, param_grid = None):
    if param_grid == None:
        param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]}

    search = GridSearchCV(estimator = Lasso(),
                          param_grid = param_grid,
                          cv = 5,
                          scoring="neg_mean_squared_error",
                          verbose=3
                          )

    search.fit(X_train_scaled, y_train)
    coefficients = search.best_estimator_.coef_
    importance = np.abs(coefficients)
    keep = np.array(X_train_scaled.columns)[importance != 0]

    return keep

In [3]:
INPUT_DIR = sys.argv[1]

In [21]:
reads_train = pd.read_csv(INPUT_DIR+'/train/readcounts_training.csv', index_col=0)
pheno_train = pd.read_csv(INPUT_DIR+'/train/pheno_training.csv', index_col=0)

In [22]:
pheno_train = pheno_train.dropna(subset=['Event', 'Event_time']) #should not try to correct these
reads_train = reads_train.T
train_ids = pheno_train.index

In [23]:
reads_test = pd.read_csv(INPUT_DIR+'/test/readcounts_test.csv', index_col=0)
pheno_test = pd.read_csv(INPUT_DIR+'/test/pheno_test.csv', index_col=0)

In [24]:
pheno_test = pheno_test.dropna(subset=['Event', 'Event_time']) #should not try to correct these
reads_test = reads_test.T
test_ids = pheno_test.index

In [25]:
reads_full = pd.concat([reads_train, reads_test])
pheno_full = pd.concat([pheno_train, pheno_test])

In [26]:
pheno_full = fillNA(pheno_full, index=pheno_full.index) #fast KNN

In [39]:
features = pd.merge(reads_full, pheno_full, left_index=True, right_index=True)
labels = features.pop('Event')
true_labels = labels.copy()
#labels.value_counts() # 0 - 4898, 1 - 445 --> highly imbalanced

X_train, y_train = features.loc[train_ids], labels[train_ids]
X_test, y_test = features.loc[test_ids], labels[test_ids]

X_train, X_test = minmax_scale(X_train, X_test)

In [42]:
metadata = [
            'Age', 
            'BodyMassIndex', 
            'BPTreatment', 
            'Smoking', 
            'PrevalentDiabetes', 
            'PrevalentHFAIL', 
            'Event_time', 
            'SystolicBP', 
            'NonHDLcholesterol', 
            'Sex'
           ]

meta_train = X_train[metadata].copy()
rds_train = X_train.drop(metadata, axis=1)

meta_test = X_test[metadata].copy()
rds_test = X_test.drop(metadata, axis=1)

In [31]:
AE_train, AE_test = run_AE(rds_train, rds_test)

/nfs/home/ababjac/base-venv/lib/python3.7/site-packages/ipykernel_launcher.py:65: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
2022-11-30 21:17:46.853314: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /sw/isaac/compilers/intel/oneAPI_2021.2.0/mpi/latest/libfabric/lib:/sw/isaac/compilers/intel/oneAPI_2021.2.0/mpi/latest/lib/release:/sw/isaac/compilers/intel/oneAPI_2021.2.0/mpi/latest/lib:/sw/isaac/compilers/intel/oneAPI_2021.2.0/ippcp/latest/lib/intel64:/sw/isaac/compilers/intel/oneAPI_2021.2.0/ipp/latest/lib/intel64:/sw/isaac/compilers/intel/oneAPI_2021.2.0/itac/latest/slib:/sw/isaac/compilers/intel/oneAPI_2021.2.0/mkl/latest/lib/intel64:/sw/isaac/compilers/intel/oneAPI_2021.2.0/

Fitting 5 folds for each of 36 candidates, totalling 180 fits


2022-11-30 21:17:46.860547: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[CV 1/5] END activation=relu, actual_dim=5749, latent_dim=100, loss=MAE, optimizer=SGD;, score=0.725 total time=   6.8s
[CV 2/5] END activation=relu, actual_dim=5749, latent_dim=100, loss=MAE, optimizer=SGD;, score=0.727 total time=   5.5s
[CV 3/5] END activation=relu, actual_dim=5749, latent_dim=100, loss=MAE, optimizer=SGD;, score=0.727 total time=   5.5s
[CV 4/5] END activation=relu, actual_dim=5749, latent_dim=100, loss=MAE, optimizer=SGD;, score=0.731 total time=   5.3s
[CV 5/5] END activation=relu, actual_dim=5749, latent_dim=100, loss=MAE, optimizer=SGD;, score=0.727 total time=   5.3s
[CV 1/5] END activation=relu, actual_dim=5749, latent_dim=100, loss=MAE, optimizer=Adam;, score=0.725 total time=   5.5s
[CV 2/5] END activation=relu, actual_dim=5749, latent_dim=100, loss=MAE, optimizer=Adam;, score=0.727 total time=   5.6s
[CV 3/5] END activation=relu, actual_dim=5749, latent_dim=100, loss=MAE, optimizer=Adam;, score=0.727 total time=   5.4s
[CV 4/5] END activation=relu, actual_

In [43]:
ET = meta_train.pop('Event_time')
important_cols = run_LASSO(meta_train, ET)
important_cols = np.append(important_cols, 'Event_time')

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END .....................alpha=0.0001;, score=-0.007 total time=   0.0s
[CV 2/5] END .....................alpha=0.0001;, score=-0.007 total time=   0.0s
[CV 3/5] END .....................alpha=0.0001;, score=-0.008 total time=   0.0s
[CV 4/5] END .....................alpha=0.0001;, score=-0.008 total time=   0.0s
[CV 5/5] END .....................alpha=0.0001;, score=-0.006 total time=   0.0s
[CV 1/5] END ......................alpha=0.001;, score=-0.007 total time=   0.0s
[CV 2/5] END ......................alpha=0.001;, score=-0.007 total time=   0.0s
[CV 3/5] END ......................alpha=0.001;, score=-0.008 total time=   0.0s
[CV 4/5] END ......................alpha=0.001;, score=-0.008 total time=   0.0s
[CV 5/5] END ......................alpha=0.001;, score=-0.006 total time=   0.0s
[CV 1/5] END .......................alpha=0.01;, score=-0.011 total time=   0.0s
[CV 2/5] END .......................alpha=0.01;, 

In [48]:
meta_train['Event_time'] = ET

In [60]:
AE_train_df = AE_train.join(meta_train[important_cols])
AE_train_df = AE_train_df.set_index(train_ids)

AE_test_df = AE_test.join(meta_test[important_cols])
AE_test_df = AE_test_df.set_index(test_ids)

In [63]:
df_AE = pd.concat([AE_train_df, AE_test_df])
df_AE['Event'] = true_labels

In [76]:
features = df_AE.copy()
times = features.pop('Event_time')
events = features.pop('Event')

train_times = times[train_ids]
train_events = events[train_ids]
y_train = make_structured_array(train_events, train_times)

test_times = times[test_ids]
test_events = events[test_ids]
y_test = make_structured_array(test_events, test_times)

X_train = features.loc[train_ids]
X_test = features.loc[test_ids]

In [77]:
est_cph_tree = GradientBoostingSurvivalAnalysis(
    n_estimators=1000, 
    random_state=8993624,
    loss='coxph',
    max_depth=5,
    max_features='sqrt',
    verbose=1
    )
est_cph_tree.fit(X_train, y_train)
est_cph_tree.score(X_test, y_test)

/nfs/home/ababjac/base-venv/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


      Iter       Train Loss   Remaining Time 
         1        2229.5295            2.89m
         2        2227.5130            2.82m
         3        2223.6770            2.85m
         4        2221.1164            2.88m
         5        2218.9751            2.92m
         6        2217.6603            2.95m
         7        2216.8221            2.95m
         8        2213.9814            2.95m
         9        2211.5565            2.95m
        10        2209.7938            2.94m
        20        2193.0224            2.89m
        30        2176.5054            2.86m
        40        2158.8742            2.82m
        50        2142.7963            2.77m
        60        2123.4308            2.73m
        70        2099.9496            2.69m
        80        2078.2823            2.65m
        90        2059.0448            2.62m
       100        2043.7171            2.59m
       200        1870.1472            2.27m
       300        1717.1228            1.98m
       40

/nfs/home/ababjac/base-venv/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


0.6062092496875106

In [78]:
OUTPUT_DIR = 'UTK_Bioinformatics_Submission_1/output/'

In [82]:
CHECK_FOLDER = os.path.isdir(OUTPUT_DIR)
if not CHECK_FOLDER:
    os.makedirs(OUTPUT_DIR)

In [98]:
y_preds = est_cph_tree.predict(X_test)

/nfs/home/ababjac/base-venv/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
y_preds = normalize(y_preds) #ensure range 0-1

In [99]:
data = {'SampleID' : test_ids, 'Score' : y_preds}
out_df = pd.DataFrame(data)
out_df.to_csv(OUTPUT_DIR+'scores.csv', index=False)